# Genetic Algorithm

A genetic algorithm starts with an initial population of solutions that evolves using selection, crossover, and mutation operators. These operators can be implemented in different ways, each affecting the performance.[](https://www.geeksforgeeks.org/dsa/genetic-algorithms/)[](https://www.mathworks.com/help/gads/how-the-genetic-algorithm-works.html)[](https://pmc.ncbi.nlm.nih.gov/articles/PMC7274742/)

## Parent Selection Methods
Parent selection determines which solutions breed and pass their qualities to the next generation.[](https://www.tutorialspoint.com/genetic_algorithms/genetic_algorithms_parent_selection.htm)
- **Roulette Wheel Selection**: Probability proportional to fitness; maintains diversity.
- **Tournament Selection**: Selects the best from random subsets; speeds convergence, possibly reduces diversity.
- **Rank Selection**: Sorts individuals by fitness and selects with a bias.
- **Elitism**: Always keeps a few best solutions; improves stability, combats loss of good genes.
- For sequencing, tournament selection is preferred for balancing exploration and exploitation.[](https://algorithmafternoon.com/books/genetic_algorithm/chapter04/)

## Crossover Methods

- **Single-point crossover**: Efficient, often used in GAs, good for simple solution encoding. Produces offspring with contiguous blocks from parents—great for maintaining solution structure.[](https://bdm.ufpa.br/bitstream/prefix/7788/1/TCC_ComparisonCrossoverOperators.pdf)[](https://wseas.com/journals/computers/2013/5705-156.pdf)
- **Multi-point crossover**: Uses multiple cut points; increases diversity and exploration capability but may disrupt good gene combinations.[](https://wseas.com/journals/computers/2013/5705-156.pdf)[](https://bdm.ufpa.br/bitstream/prefix/7788/1/TCC_ComparisonCrossoverOperators.pdf)
- **Uniform crossover**: Swaps genes randomly based on a mask, promoting high diversity but can break useful gene combinations more easily.[](https://bdm.ufpa.br/bitstream/prefix/7788/1/TCC_ComparisonCrossoverOperators.pdf)[](https://wseas.com/journals/computers/2013/5705-156.pdf)
- **Specialized crossovers** (Order Crossover OX, Partially Mapped PMX, Cycle Crossover CX): Designed for sequencing problems and maintain order or position relationships, typically yielding better results for job sequencing tasks.[](https://dergipark.org.tr/en/download/article-file/4353944)[](https://www.atlantis-press.com/article/25845582.pdf)[](https://wseas.com/journals/computers/2013/5705-156.pdf)
	- For job sequencing, Order and PMX crossovers are commonly recommended due to their ability to preserve job order and avoid duplicates.[](https://www.atlantis-press.com/article/25845582.pdf)[](https://dergipark.org.tr/en/download/article-file/4353944)

## Mutation Operators
Mutation introduces random changes to individual solutions to prevent premature convergence and explore new search regions. Typical mutation methods for job sequencing include:[](https://en.wikipedia.org/wiki/Genetic_algorithm)

- **Swap Mutation**: Randomly swaps two jobs in the sequence; maintains feasibility.
- **Insertion Mutation**: Moves a job to a random position; helps maintain legal sequences.
- Mutation rate must be balanced: too low may cause genetic drift, too high can erase good solutions unless elite retention is used.[](https://en.wikipedia.org/wiki/Genetic_algorithm)

In [31]:
import random

## Fitness function

In [ ]:
def fitness(schedule, deadlines, profits):
    total_profit = 0
    job_count = 0

    i = 0
    for job in schedule:
        if i < deadlines[job]:
            total_profit += profits[job]
            job_count += 1
        i += 1

    return total_profit, job_count

## Population

In [33]:
def init_pop(n_jobs, pop_size):
    population = []

    for _ in range(pop_size):
        candidate = list(range(n_jobs))
        random.shuffle(candidate)
        population.append(candidate)

    return population

## Selection

In [34]:
def tournament_selection(pop, scores, k=3):
    selected = random.sample(range(len(pop)), k)
    selected.sort(key=lambda i: scores[i], reverse=True)

    return pop[selected[0]]

def roulette_wheel_selection(pop, scores):
    total_fitness = sum(scores)
    
    if total_fitness == 0:
        return random.choice(pop)
    
    pick = random.uniform(0, total_fitness)
    current = 0
    
    for i, individual in enumerate(pop):
        current += scores[i]
        if current > pick:
            return individual
    
    return pop[-1]
    
def rank_selection(pop, scores):
    sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i])
    ranks = [0] * len(scores)
    for i, idx in enumerate(sorted_indices):
        ranks[idx] = i + 1
    
    return roulette_wheel_selection(pop, ranks)

## Crossover

In [35]:
# standard single point does not work here because it can end up making schedules with duplicate and missing jobs
def order_preserving_one_point_crossover(parent1, parent2):
    n = len(parent1)
    cutoff = random.randrange(1, n)
    child = parent1[:cutoff]
    for gene in parent2:
        if gene not in child:
            child.append(gene)
    return child

def uniform_crossover(parent1, parent2, prob=0.5):
    n = len(parent1)
    child = [-1] * n
    used = set()
    
    for i in range(n):
        if random.random() < prob:
            if parent1[i] not in used:
                child[i] = parent1[i]
                used.add(parent1[i])
        else:
            if parent2[i] not in used:
                child[i] = parent2[i]
                used.add(parent2[i])
    
    missing = [gene for gene in range(n) if gene not in used]
    for i in range(n):
        if child[i] == -1:
            child[i] = missing.pop(0)
    
    return child

def order_crossover(parent1, parent2):
    n = len(parent1)
    p1, p2 = sorted(random.sample(range(n), 2))
    child = [-1] * n
    child[p1:p2] = parent1[p1:p2]
    ptr = p2

    for idx in range(n):
        if parent2[idx] not in child:
            if ptr == n: ptr = 0
            child[ptr] = parent2[idx]
            ptr += 1

    return child

def partially_mapped_crossover(parent1, parent2):
    n = len(parent1)
    p1, p2 = sorted(random.sample(range(n), 2))
    
    child = [-1] * n
    child[p1:p2] = parent1[p1:p2]
    
    mapping = {}
    for i in range(p1, p2):
        if parent2[i] not in child:
            mapping[parent2[i]] = parent1[i]
    
    for i in range(n):
        if child[i] == -1:
            val = parent2[i]
            while val in mapping:
                val = mapping[val]
            child[i] = val
    
    return child

def cycle_crossover(parent1, parent2):
    n = len(parent1)
    child = [-1] * n
    visited = [False] * n
    
    for start in range(n):
        if visited[start]:
            continue
            
        current = start
        cycle_positions = []
        
        while not visited[current]:
            visited[current] = True
            cycle_positions.append(current)
            
            value = parent1[current]
            current = parent2.index(value)
        
        use_parent1 = (start % 2 == 0) 
        for pos in cycle_positions:
            child[pos] = parent1[pos] if use_parent1 else parent2[pos]
    
    return child

## Mutation

In [36]:
def swap_mutation(chrom):
    a, b = random.sample(range(len(chrom)), 2)
    chrom[a], chrom[b] = chrom[b], chrom[a]

    return chrom

def insertion_mutation(chrom):
    chrom = chrom[:]
    if len(chrom) < 2:
        return chrom
    
    i = random.randrange(len(chrom))
    j = random.randrange(len(chrom))
    gene = chrom.pop(i)
    chrom.insert(j, gene)
    return chrom

## Full implementation

In [37]:
def genetic_algorithm(deadline, profit, pop_size=50, num_gens=200, crossover_rate=0.7, mutation_rate=0.1):
    n_jobs = len(deadline)
    population = init_pop(n_jobs, pop_size)
    best_profit = 0
    best_schedule = None

    for gen in range(num_gens):
        scores = [fitness(ind, deadline, profit)[0] for ind in population]
        new_pop = []

        for _ in range(pop_size):
            parent1 = tournament_selection(population, scores)
            parent2 = tournament_selection(population, scores)

            if random.random() < crossover_rate:
                child = order_crossover(parent1, parent2)
            else:
                child = parent1[:]

            if random.random() < mutation_rate:
                child = swap_mutation(child)

            new_pop.append(child)

        population = new_pop

        for ind in population:
            score, jobs_used = fitness(ind, deadline, profit)

            if score > best_profit:
                best_profit = score
                best_schedule = ind

    jobs_completed = fitness(best_schedule, deadline, profit)[1]

    return jobs_completed, best_profit, best_schedule

In [38]:
def genetic_algorithm_configurable(deadline, profit,
                                   pop_size=50, num_gens=100,
                                   crossover_rate=0.7, mutation_rate=0.1,
                                   selection_method="tournament",
                                   crossover_method="order_x",
                                   mutation_method="swap",
                                   elitism=0):
    
    selection_funcs = {
        "tournament": tournament_selection,
        "roulette": roulette_wheel_selection,
        "rank": rank_selection
    }
    
    crossover_funcs = {
        "order_x": order_crossover,
        "pmxorder_x": partially_mapped_crossover,
        "cycle": cycle_crossover,
        "uniform": uniform_crossover,
        "single_point": order_preserving_one_point_crossover
    }
    
    mutation_funcs = {
        "swap": swap_mutation,
        "insertion": insertion_mutation
    }
    
    n_jobs = len(deadline)
    population = init_pop(n_jobs, pop_size)
    best_profit = 0
    best_schedule = None
    
    selection_func = selection_funcs[selection_method]
    crossover_func = crossover_funcs[crossover_method]
    mutation_func = mutation_funcs[mutation_method]

    for gen in range(num_gens):
        scores = [fitness(ind, deadline, profit)[0] for ind in population]
        
        current_best = max(scores) if scores else 0
        
        if current_best > best_profit:
            best_profit = current_best
            best_idx = scores.index(current_best)
            best_schedule = population[best_idx][:]
        
        new_pop = []
        
        if elitism > 0 and scores:
            elite_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:elitism]
            for idx in elite_indices:
                new_pop.append(population[idx][:])
        
        for _ in range(pop_size - elitism):
            parent1 = selection_func(population, scores)
            parent2 = selection_func(population, scores)

            if random.random() < crossover_rate:
                child = crossover_func(parent1, parent2)
            else:
                child = parent1[:]

            if random.random() < mutation_rate:
                child = mutation_func(child)

            new_pop.append(child)

        population = new_pop

    jobs_completed = fitness(best_schedule, deadline, profit)[1]
    return jobs_completed, best_profit, best_schedule

## Usage

In [39]:
deadline = [2, 1, 2, 1, 1]
profit = [100, 19, 27, 25, 15]
# ans = genetic_algorithm(deadline, profit)
# print(ans)

genetic_algorithm_configurable(deadline, profit)

(2, 127, [2, 0, 4, 1, 3])

## Evaluation

In [40]:
# evaluation here